In [1]:
import torch
from torchvision import transforms

from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import os 
from tqdm import tqdm

from pose_crop import PoseCrop

In [2]:
def load_model():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = torch.load('yolov7-w6-pose.pt', map_location=device)['model']
    # Put in inference mode
    model.float().eval()

    if torch.cuda.is_available():
        # half() turns predictions into float16 tensors
        # which significantly lowers inference time
        model.half().to(device)
    return model

model = load_model()

In [12]:
def run_inference(url):
    image = cv2.imread(url) # shape: (480, 640, 3)
    # Resize and pad image
    image = letterbox(image, 960, stride=64, auto=True)[0] # shape: (768, 960, 3)
    # Apply transforms
    image = transforms.ToTensor()(image) # torch.Size([3, 768, 960])
    # Turn image into batch
    image = image.unsqueeze(0) # torch.Size([1, 3, 768, 960])
    output, _ = model(image) # torch.Size([1, 45900, 57])
    return output, image

In [4]:
# read the dataset csv
df = pd.read_csv('data_final.csv')

In [5]:
filenames = df['img'].values
filenames.shape

(9516,)

In [6]:
images_dir = '/Users/zubayer/codes/michiluki/images/'

In [13]:
# create 5 empty pandas dataframes
chest_df = pd.DataFrame(columns=['img', 'value'])
hip_df = pd.DataFrame(columns=['img', 'value'])
head_df = pd.DataFrame(columns=['img', 'hair_black', 'hair_red', 'hair_blonde'])
eye_df = pd.DataFrame(columns=['img', 'value'])
feet_df = pd.DataFrame(columns=['img', 'value'])

ps = PoseCrop()

for files in tqdm(filenames):
    url = os.path.join(images_dir, files)
    output, image = run_inference(url)
    # Apply NMS
    output = non_max_suppression_kpt(output, 
                                     0.25, # Confidence Threshold
                                     0.65, # IoU Threshold
                                     nc=model.yaml['nc'], # Number of Classes
                                     nkpt=model.yaml['nkpt'], # Number of Keypoints
                                     kpt_label=True)

    with torch.no_grad():
        output = output_to_keypoint(output)

    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)

    if len(output) > 1:
        continue


    chest_img = ps.get_chest_image(nimg, output)
    hip_img = ps.get_hip_image(nimg, output)
    head_img = ps.get_head_image(nimg, output)
    eye_img = ps.get_eye_image(nimg, output)
    left_foot_img, right_foot_img = ps.get_feet_image(nimg, output)

    if chest_img is not None:
        cv2.imwrite('dataset/chests/' + files, chest_img)
        # check the value of img in df of 'object_naked' column
        chest_value = df[df['img'] == files]['object_naked'].values[0]
        # row = pd.DataFrame({'img': files, 'value': chest_value}, index=[0])
        row = [files, chest_value]
        # chest_df = chest_df.append({'img': files, 'value': chest_value})
        chest_df.loc[len(chest_df)] = row

    if hip_img is not None:
        cv2.imwrite('dataset/hips/' + files, hip_img)
        # check the value of img in df of 'object_naked' column
        hip_value = df[df['img'] == files]['object_naked'].values[0]
        row = [files, hip_value]
        # hip_df = hip_df.append({'img': files, 'value': hip_value}, ignore_index=True)
        hip_df.loc[len(hip_df)] = row

    if head_img is not None:
        cv2.imwrite('dataset/heads/' + files, head_img)
        hair_black = df[df['img'] == files]['hair_black'].values[0]
        hair_red = df[df['img'] == files]['hair_red'].values[0]
        hair_blonde = df[df['img'] == files]['hair_blonde'].values[0]
        row = [files, hair_black, hair_red, hair_blonde]
        # head_df = head_df.append({'img': files, 'hair_black': hair_black, 'hair_red': hair_red, 'hair_blonde': hair_blonde}, ignore_index=True)
        head_df.loc[len(head_df)] = row


    if eye_img is not None:
        cv2.imwrite('dataset/eyes/' + files, eye_img)
        # check the value of img in df of 'object_glasses' column
        eye_value = df[df['img'] == files]['object_glasses'].values[0]
        row = [files, eye_value]
        # eye_df = eye_df.append({'img': files, 'value': eye_value}, ignore_index=True)
        eye_df.loc[len(eye_df)] = row

    if left_foot_img is not None:
        cv2.imwrite('dataset/feet/' + files, left_foot_img)
        # check the value of img in df of 'object_heels' column
        feet_value = df[df['img'] == files]['object_heels'].values[0]
        file_name = files.replace('.', 'left.')
        row = [file_name, feet_value]
        # feet_df = feet_df.append({'img': file_name, 'value': feet_value}, ignore_index=True)
        feet_df.loc[len(feet_df)] = row

    if right_foot_img is not None:
        cv2.imwrite('dataset/feet/' + files, right_foot_img)
        # check the value of img in df of 'object_heels' column
        feet_value = df[df['img'] == files]['object_heels'].values[0]
        file_name = files.replace('.', 'right.')
        row = [file_name, feet_value]
        # feet_df = feet_df.append(row, ignore_index=True)
        feet_df.loc[len(feet_df)] = row
        

chest_df.to_csv('dataset/chest.csv', index=False)
hip_df.to_csv('dataset/hip.csv', index=False)
head_df.to_csv('dataset/head.csv', index=False)
eye_df.to_csv('dataset/eye.csv', index=False)
feet_df.to_csv('dataset/feet.csv', index=False)





  0%|          | 5/9516 [00:22<11:51:38,  4.49s/it]


KeyboardInterrupt: 

In [10]:
left_foot_img

array([], shape=(0, 243, 3), dtype=uint8)

In [ ]:
output, image = run_inference('/Users/zubayer/codes/michiluki/images/2022-09-12_11-19-43_UTC_1.jpg')

output = non_max_suppression_kpt(output, 
                                     0.25, # Confidence Threshold
                                     0.65, # IoU Threshold
                                     nc=model.yaml['nc'], # Number of Classes
                                     nkpt=model.yaml['nkpt'], # Number of Keypoints
                                     kpt_label=True)

with torch.no_grad():
    output = output_to_keypoint(output)

nimg = image[0].permute(1, 2, 0) * 255
nimg = nimg.cpu().numpy().astype(np.uint8)